In [11]:
# evaluate a smoothed classifier on a dataset
import os
import setGPU
from time import time
import torch
import datetime
from torchvision import transforms
from torch.utils.data import DataLoader
import numpy as np

from iCIFAR100 import iCIFAR100
from myNetwork import network
from ResNet import resnet18_cbam


classes=[0, 20]
sigma = 0.25
# outfile = "predict/a_model1_09.txt"
batch = 100
skip = 1
max = -1
N0 = 100
N = 1000
alpha = 0.001

feature_extractor=resnet18_cbam()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if __name__ == "__main__":

    numclass=10

    # load the base classifier
    # base_classifier = network(numclass,feature_extractor).to(device)
    base_classifier = torch.load('model/accuracy_73.900_KNN_accuracy_76.050_increment_29_net.pt').to(device)

    # load the dataset
    test_transform = transforms.Compose([#transforms.Resize(img_size),
                                        transforms.ToTensor(),
                                        transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))])
    dataset = iCIFAR100('dataset', train=False, transform=test_transform, download=True)
    dataset.getTestData(classes)
    dataset = DataLoader(dataset=dataset,
                                 shuffle=True,
                                 batch_size=1)

    # create the smooothed classifier g
    # smoothed_classifier = Smooth(base_classifier, numclass, sigma)

    # prepare output file
    # f = open(outfile, 'w')
    # print("idx\tlabel\tpredict\tcorrect\ttime", file=f, flush=True)
    res = [0, 0]
    base_classifier.eval()

    # iterate through the dataset
    for i, (indexs, imgs, labels) in enumerate(dataset):

        # only certify every args.skip examples, and stop after args.max examples
        if i % skip != 0:
            continue
        if i == max:
            break
        
        x, label = imgs.to(device), labels.to(device)
        # before_time = time()

        # make the prediction
        
        with torch.no_grad():
            batch = x.repeat((1, 1, 1, 1))
            noise = torch.randn_like(batch, device='cuda') * sigma
            prediction = base_classifier(batch + noise).argmax(1)
        # after_time = time()
        correct = int(prediction.cpu() == label.cpu())

        # res.append([prediction, label])
        if label.cpu() < 10: res[0] += correct
        else: res[1] += correct

        # time_elapsed = str(datetime.timedelta(seconds=(after_time - before_time)))
        # print("{}\t{}\t{}\t{}\t{}".format(i, label, prediction, correct, time_elapsed), file=f, flush=True)

    # f.close()
    print(res)


Files already downloaded and verified
the size of test set is (2000, 32, 32, 3)
the size of test label is (2000,)
[702, 517]
